In [1]:
!python -V

Python 3.12.1


In [2]:
import pandas as pd
import pickle


from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [3]:
df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')
print('Q1: ', len(df.columns))

Q1:  19


In [4]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

print('Q2: ', df['duration'].std())

Q2:  42.59435124195458


In [5]:
before_drop = len(df)
df = df[(df.duration >= 1) & (df.duration <= 60)]
after_drop = len(df)
print('Q3: ', after_drop/before_drop)

Q3:  0.9812202822125979


In [6]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)

In [7]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
with open('dv.pkl','wb') as f:
    pickle.dump(dv,f)
print('Q4: ', X_train.shape[1])

Q4:  515


In [8]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

with open('model.pkl','wb') as f:
    pickle.dump(lr,f)

print('Q5: ', root_mean_squared_error(y_train, y_pred))

Q5:  7.649261932106969


In [9]:
# with open('model.pkl', 'rb') as f:
#     lr = pickle.load(f)

# with open('dv.pkl', 'rb') as f:
#     dv = pickle.load(f)

df_test = pd.read_parquet('data/yellow_tripdata_2023-02.parquet')
df_test['duration'] = df_test.tpep_dropoff_datetime - df_test.tpep_pickup_datetime
df_test.duration = df_test.duration.apply(lambda td: td.total_seconds() / 60)
categorical = ['PULocationID', 'DOLocationID']
df_test[categorical] = df_test[categorical].astype(str)

In [10]:
test_dicts = df_test[categorical].to_dict(orient='records')
X_test = dv.transform(test_dicts)
y_pred = lr.predict(X_test)
y_test = df_test["duration"].values
print('Q6: ', root_mean_squared_error(y_test, y_pred))

Q6:  42.28477577694284
